In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import mmread, mmwrite

In [3]:
# tmp = pd.Index(gex['demuxlet'].unique()).drop(['unassigned', 'doublet'])
# pd.DataFrame(tmp).to_csv(f'{projdir}/txt/all_donors.txt', sep='\t', header=None, index=None)

In [4]:
projdir = '/u/home/t/terencew/project-cluo/demux_benchmark/adelus_2023/'
# donors = list(np.loadtxt(f'{projdir}/txt/donors.txt', dtype=str))
donors = list(np.loadtxt(f'{projdir}/txt/all_donors.txt', dtype=str))
samples = list(np.loadtxt(f'{projdir}/txt/samples.txt', dtype=str))
new_samples = [x.replace('MT_', 'MT-').replace('pooled_','pooled-') for x in samples]
s = samples[0]

gex_methods = ['demuxlet', 'freemuxlet', 'vireo', 'vireo_nogenos',
               'scsplit', 'scsplit_nogenos',
    'souporcell', 'souporcell_nogenos', 'demuxalot', 'ambimux']
atac_methods = ['demuxlet', 'freemuxlet',  'vireo', 'vireo_nogenos',
               'scsplit', 'scsplit_nogenos',
    'souporcell', 'souporcell_nogenos', 'scavengers', 'ambimux']

con_cols = ['C1', 'C2', 'I1', 'I2']

In [5]:
!ls -lt {projdir}/csv/demux/inter_gex.csv

-rw-r--r--. 1 terencew cluo 3463167 Mar 21 03:15 /u/home/t/terencew/project-cluo/demux_benchmark/adelus_2023//csv/demux/inter_gex.csv


In [6]:
h_donors = list(np.loadtxt(f'{projdir}/txt/all_donors.txt', dtype=str))
mt13_donors = list(np.loadtxt(f'{projdir}/txt/MT1-3_donors.txt', dtype=str))
mt4_donors = list(np.loadtxt(f'{projdir}/txt/MT4_donors.txt', dtype=str))
mt5_donors = list(np.loadtxt(f'{projdir}/txt/MT5_donors.txt', dtype=str))

In [7]:
replace_dict = {'pooledendMT1' : 'pooled_endMT_1',
               'pooledendMT2' : 'pooled_endMT_2',
                'pooledendMT3' : 'pooled_endMT_3',
                'pooledendMT4' : 'pooled_endMT_4',
                'pooledendMT5' : 'pooled_endMT_5'}

In [8]:
!ls {projdir}/csv/demux/

ambimux.csv		     inter_atac_raw_donors.csv	 singlet_gex.csv
atac			     inter_gex.csv		 singlet_merged.csv
dblfinder		     inter_gex_fixed_donors.csv  soup_ambient.csv
gex			     inter_gex_raw_donors.csv	 status_atac.csv
inter_atac.csv		     known_soup_ambient.csv	 status_gex.csv
inter_atac_fixed_donors.csv  singlet_atac.csv		 status_merged.csv


In [23]:
gex = pd.read_csv(f'{projdir}/csv/demux/inter_gex_raw_donors.csv', sep='\t',
                  header=0, index_col=0)
atac = pd.read_csv(f'{projdir}/csv/demux/inter_atac_raw_donors.csv', sep='\t',
                   header=0, index_col=0)

gex.shape, atac.shape

((35201, 9), (35201, 9))

In [24]:
ambimux = pd.read_csv(f'{projdir}/csv/demux/ambimux.csv', sep='\t', header=0, index_col=0)
gex['ambimux'] = ambimux['best_sample']
atac['ambimux'] = ambimux['best_sample']

In [25]:
tmp_gex = gex.copy()
tmp_atac = atac.copy()
tmp_gex['inter'] = gex.eq(tmp_gex.iloc[:, 0], axis=0).all(1)
tmp_atac['inter'] = atac.eq(tmp_atac.iloc[:, 0], axis=0).all(1)
# tmp_gex['majority'] = gex.mode(axis=1)[0]
# tmp_atac['majority'] = atac.mode(axis=1)[0]

# gex['majority'] = tmp_gex['majority']
gex['inter'] = tmp_gex['inter']
# atac['majority'] = tmp_atac['majority']
atac['inter'] = tmp_atac['inter']

In [26]:
gex['sample'] = [''.join(x.split('_')[1:]) for x in gex.index]
atac['sample'] = [''.join(x.split('_')[1:]) for x in atac.index]

gex['sample'].replace(replace_dict, inplace=True)
atac['sample'].replace(replace_dict, inplace=True)

In [27]:
gex.head()

,demuxlet,freemuxlet,vireo,vireo_nogenos,souporcell,souporcell_nogenos,demuxalot,scsplit,scsplit_nogenos,ambimux,inter,sample
AAACAGCCAACTAGCC-1_ctrl6h,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,True,ctrl6h
AAACAGCCACACAATT-1_ctrl6h,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,True,ctrl6h
AAACAGCCACTCGCTC-1_ctrl6h,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,doublet,U114.rep0.uD563_U114.rep0.uD563,U115.rep0.uD557_U115.rep0.uD557,False,ctrl6h
AAACCAACAGAACCGA-1_ctrl6h,J98.rep0.uD9_J98.rep0.uD9,J98.rep0.uD9_J98.rep0.uD9,J98.rep0.uD9_J98.rep0.uD9,J98.rep0.uD9_J98.rep0.uD9,J98.rep0.uD9_J98.rep0.uD9,J98.rep0.uD9_J98.rep0.uD9,doublet,T100.rep0.uD20_T100.rep0.uD20,U114.rep0.uD563_U114.rep0.uD563,J98.rep0.uD9_J98.rep0.uD9,False,ctrl6h
AAACCGAAGCTATGAC-1_ctrl6h,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,V101.rep0.uD22_V101.rep0.uD22,V101.rep0.uD22_V101.rep0.uD22,U114.rep0.uD563_U114.rep0.uD563,False,ctrl6h


In [28]:
def read_varcon_steps(indir, steps, cov_thresh):
    step = steps[0]
    con_path = f'{indir}/batches/{step}_cov{cov_thresh}_con_counts.csv'
    varcon = pd.read_csv(con_path, sep='\t', header=0, index_col=0)
    for step in steps[1:]:
        con_path = f'{indir}/batches/{step}_cov{cov_thresh}_con_counts.csv'
        tmp_con = pd.read_csv(con_path, sep='\t', header=0, index_col=0)
        varcon = pd.concat([varcon, tmp_con])
    return varcon

In [29]:
def process_varcon(demux, varcon, donors, method):
    cols = ['C1', 'C2', 'I1', 'I2', 'donor']
    counts = pd.DataFrame(columns=cols)
    for donor in donors:
        bc_mask = demux[method] == donor
        tmp_assign = demux[bc_mask]
        tmp_con = varcon[bc_mask]
#         col_mask = [donor in x for x in tmp_con.columns]
        col_mask = [x[:-3] == donor for x in tmp_con.columns]
        select_cols = tmp_con.columns[col_mask]
        tmp_con = tmp_con[select_cols]
        tmp_con.columns = ['C1', 'C2', 'I1', 'I2']
        tmp_con['donor'] = donor
        counts = pd.concat([counts, tmp_con])
    return counts

In [30]:
interval = 100
cov_thresh = 20
s = samples[0]

indir = f'{projdir}/demux/regular/var_consistency/gex/{s}/'
n_drops = sum(gex['sample'] == s)
steps = list(range(0, n_drops, interval))

gex_con = read_varcon_steps(indir, steps, cov_thresh)
gex_con.index = [f'{x}_{s}' for x in gex_con.index]
gex_con.shape

for s in samples[1:]:
    print(s)
    indir = f'{projdir}/demux/regular/var_consistency/gex/{s}/'
    n_drops = sum(gex['sample'] == s)
    steps = list(range(0, n_drops, interval))
    tmp_gex_con = read_varcon_steps(indir, steps, cov_thresh)
    tmp_gex_con.index = [f'{x}_{s}' for x in tmp_gex_con.index]
    gex_con = pd.concat([gex_con, tmp_gex_con])
    
gex_con.shape

il1b6h
pooled_endMT_1
pooled_endMT_2
pooled_endMT_3
pooled_endMT_4
pooled_endMT_5


(35201, 52)

In [31]:
gex['vireo'].value_counts()

J98.rep0.uD9                       5934
TELOS                              5015
U114.rep0.uD563                    4547
V105.rep1.uD60                     4040
doublet                            2911
U114.rep0.uD563_U114.rep0.uD563    2202
unassigned                         2147
V101.rep0.uD22_V101.rep0.uD22      1799
M103.rep0.uD62                     1614
T100.rep0.uD20_T100.rep0.uD20      1138
J76.rep2.uD52                      1010
J98.rep0.uD9_J98.rep0.uD9           975
D120.rep0.uD562_D120.rep0.uD562     937
D120.rep0.uD562                     681
U115.rep0.uD557_U115.rep0.uD557     251
Name: vireo, dtype: int64

In [38]:
method = 'ambimux'
tmp_ambimux_gex = process_varcon(gex, gex_con, donors, method)

In [43]:
tmp_ambimux_gex.to_csv(f'{projdir}/csv/var_consistency/gex/merged/ambimux_cov20_varcon.csv',
                       sep='\t', header=True, index=True)

In [36]:
interval = 100
cov_thresh = 20
s = samples[0]

indir = f'{projdir}/demux/regular/var_consistency/atac/{s}/'
n_drops = sum(atac['sample'] == s)
steps = list(range(0, n_drops, interval))

atac_con = read_varcon_steps(indir, steps, cov_thresh)
atac_con.index = [f'{x}_{s}' for x in atac_con.index]
atac_con.shape

for s in samples[1:]:
    print(s)
    indir = f'{projdir}/demux/regular/var_consistency/atac/{s}/'
    n_drops = sum(atac['sample'] == s)
    steps = list(range(0, n_drops, interval))
    tmp_atac_con = read_varcon_steps(indir, steps, cov_thresh)
    tmp_atac_con.index = [f'{x}_{s}' for x in tmp_atac_con.index]
    atac_con = pd.concat([atac_con, tmp_atac_con])
    
atac_con.shape

il1b6h
pooled_endMT_1
pooled_endMT_2
pooled_endMT_3
pooled_endMT_4
pooled_endMT_5


(35201, 52)

In [40]:
method = 'ambimux'
tmp_ambimux_atac = process_varcon(atac, atac_con, donors, method)

In [42]:
tmp_ambimux_atac.to_csv(f'{projdir}/csv/var_consistency/atac/merged/ambimux_cov20_varcon.csv',
                       sep='\t', header=True, index=True)

In [18]:
interval = 100
cov_thresh = 20
s = samples[0]

indir = f'{projdir}/demux/regular/var_consistency/gex/{s}/'
n_drops = sum(gex['sample'] == s)
steps = list(range(0, n_drops, interval))

gex_con = read_varcon_steps(indir, steps, cov_thresh)
gex_con.index = [f'{x}_{s}' for x in gex_con.index]
gex_con.shape

for s in samples[1:]:
    print(s)
    indir = f'{projdir}/demux/regular/var_consistency/gex/{s}/'
    n_drops = sum(gex['sample'] == s)
    steps = list(range(0, n_drops, interval))
    tmp_gex_con = read_varcon_steps(indir, steps, cov_thresh)
    tmp_gex_con.index = [f'{x}_{s}' for x in tmp_gex_con.index]
    gex_con = pd.concat([gex_con, tmp_gex_con])
    
gex_con.shape

il1b6h
pooled_endMT_1
pooled_endMT_2
pooled_endMT_3
pooled_endMT_4
pooled_endMT_5


(35201, 52)

In [22]:
gex

,demuxlet,freemuxlet,vireo,vireo_nogenos,souporcell,souporcell_nogenos,demuxalot,scsplit,scsplit_nogenos,majority,inter,sample
AAACAGCCAACTAGCC-1_ctrl6h,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,True,ctrl6h
AAACAGCCACACAATT-1_ctrl6h,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,D120.rep0.uD562_D120.rep0.uD562,True,ctrl6h
AAACAGCCACTCGCTC-1_ctrl6h,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,U115.rep0.uD557_U115.rep0.uD557,doublet,U114.rep0.uD563_U114.rep0.uD563,U115.rep0.uD557_U115.rep0.uD557,False,ctrl6h
AAACCAACAGAACCGA-1_ctrl6h,J98.rep0.uD9_J98.rep0.uD9,J98.rep0.uD9_J98.rep0.uD9,J98.rep0.uD9_J98.rep0.uD9,J98.rep0.uD9_J98.rep0.uD9,J98.rep0.uD9_J98.rep0.uD9,J98.rep0.uD9_J98.rep0.uD9,doublet,T100.rep0.uD20_T100.rep0.uD20,U114.rep0.uD563_U114.rep0.uD563,J98.rep0.uD9_J98.rep0.uD9,False,ctrl6h
AAACCGAAGCTATGAC-1_ctrl6h,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,U114.rep0.uD563_U114.rep0.uD563,V101.rep0.uD22_V101.rep0.uD22,V101.rep0.uD22_V101.rep0.uD22,U114.rep0.uD563_U114.rep0.uD563,False,ctrl6h
...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTGTTCGGGATTT-1_pooled_endMT_5,V105.rep1.uD60,V105.rep1.uD60,V105.rep1.uD60,V105.rep1.uD60,V105.rep1.uD60,V105.rep1.uD60,V105.rep1.uD60,V105.rep1.uD60,V105.rep1.uD60,V105.rep1.uD60,True,pooled_endMT_5
TTTGTGTTCGTTATAG-1_pooled_endMT_5,doublet,TELOS,TELOS,TELOS,TELOS,TELOS,TELOS,U114.rep0.uD563,U114.rep0.uD563,TELOS,False,pooled_endMT_5
TTTGTTGGTCCCGAAG-1_pooled_endMT_5,unassigned,TELOS,unassigned,TELOS,TELOS,TELOS,TELOS,TELOS,TELOS,TELOS,False,pooled_endMT_5
TTTGTTGGTGAGACTC-1_pooled_endMT_5,U114.rep0.uD563,U114.rep0.uD563,U114.rep0.uD563,U114.rep0.uD563,U114.rep0.uD563,U114.rep0.uD563,U114.rep0.uD563,U114.rep0.uD563,U114.rep0.uD563,U114.rep0.uD563,True,pooled_endMT_5


In [20]:
cov_thresh = [0, 10, 20]
outdir = f'{projdir}/csv/var_consistency/gex/merged'

for cov in cov_thresh:
    print(cov)
    s = samples[0]
    indir = f'{projdir}/demux/regular/var_consistency/gex/{s}/'
    n_drops = sum(gex['sample'] == s)
    steps = list(range(0, n_drops, interval))

    gex_con = read_varcon_steps(indir, steps, cov)
    gex_con.index = [f'{x}_{s}' for x in gex_con.index]
    gex_con.shape

    for s in samples[1:]:
        indir = f'{projdir}/demux/regular/var_consistency/gex/{s}/'
        n_drops = sum(gex['sample'] == s)
        steps = list(range(0, n_drops, interval))
        tmp_gex_con = read_varcon_steps(indir, steps, cov)
        tmp_gex_con.index = [f'{x}_{s}' for x in tmp_gex_con.index]
        gex_con = pd.concat([gex_con, tmp_gex_con])
        
    for method in gex_methods:
        consistency = process_varcon(gex, gex_con, donors, method)
        consistency.to_csv(f'{outdir}/{method}_cov{cov}_varcon.csv', sep='\t', header=True, index=True)
    
    mask = gex_con.index.duplicated()
    gex_con = gex_con[~mask]
    print(gex.shape, gex_con.shape)
    
    method = 'majority'
    consistency = process_varcon(gex, gex_con, donors, method)
    consistency.to_csv(f'{outdir}/{method}_cov{cov}_varcon.csv', sep='\t', header=True, index=True)
    
    method = 'inter'
    mask = gex['inter']
    tmp_gex = pd.DataFrame(gex[mask].iloc[:,0])
    tmp_gex_con = gex_con[mask]
    tmp_gex.columns = ['inter']
    consistency = process_varcon(tmp_gex, tmp_gex_con, donors, method)
    consistency.to_csv(f'{outdir}/{method}_cov{cov}_varcon.csv', sep='\t', header=True, index=True)

0


KeyError: 'ambimux'

In [ ]:
cov_thresh = [0, 10, 20]
outdir = f'{projdir}/csv/var_consistency/real/{s}'
# indir = f'{projdir}/ambient/var_consistency/atac/{s}/'
indir = f'{projdir}/demux/regular/var_consistency/atac/{s}/'

for cov in cov_thresh:
    print(cov)
    s = samples[0]
    indir = f'{projdir}/demux/regular/var_consistency/atac/{s}/'
    n_drops = sum(atac['sample'] == s)
    steps = list(range(0, n_drops, interval))

    atac_con = read_varcon_steps(indir, steps, cov)
    atac_con.index = [f'{x}_{s}' for x in atac_con.index]
    atac_con.shape

    for s in samples[1:]:
        indir = f'{projdir}/demux/regular/var_consistency/atac/{s}/'
        n_drops = sum(atac['sample'] == s)
        steps = list(range(0, n_drops, interval))
        tmp_atac_con = read_varcon_steps(indir, steps, cov)
        tmp_atac_con.index = [f'{x}_{s}' for x in tmp_atac_con.index]
        atac_con = pd.concat([atac_con, tmp_atac_con])
        
    for method in atac_methods:
        outdir = f'{projdir}/csv/var_consistency/atac/merged'
        consistency = process_varcon(atac, atac_con, donors, method)
        consistency.to_csv(f'{outdir}/{method}_cov{cov}_varcon.csv', sep='\t', header=True, index=True)
    
    mask = atac_con.index.duplicated()
    atac_con = atac_con[~mask]
    print(atac.shape, atac_con.shape)
    
    method = 'majority'
    consistency = process_varcon(atac, atac_con, donors, method)
    consistency.to_csv(f'{outdir}/{method}_cov{cov}_varcon.csv', sep='\t', header=True, index=True)
    
    method = 'inter'
    mask = atac['inter']
    tmp_atac = pd.DataFrame(atac[mask].iloc[:,0])
    tmp_atac_con = atac_con[mask]
    tmp_atac.columns = ['inter']
    consistency = process_varcon(tmp_atac, tmp_atac_con, donors, method)
    consistency.to_csv(f'{outdir}/{method}_cov{cov}_varcon.csv', sep='\t', header=True, index=True)

In [ ]:
# cov_thresh = [0, 10, 20]
# interval = 100

# for i,s in enumerate(new_samples):
#     print(s)
#     if s in ['ctrl6h', 'il1b6h']:
#         donors = h_donors.copy()
#     elif s in ['pooled-endMT-1', 'pooled-endMT-2', 'pooled-endMT-3']:
#         donors = mt13_donors.copy()
#     elif s == 'pooled-endMT-4':
#         donors = mt4_donors.copy()
#     elif s == 'pooled-endMT-5':
#         donors = mt5_donors.copy()

#     gex = pd.read_csv(f'{projdir}/csv/demux/inter_gex_raw_donors.csv', sep='\t',
#                   header=0, index_col=0)

#     atac = pd.read_csv(f'{projdir}/csv/demux/inter_atac_raw_donors.csv', sep='\t',
#                        header=0, index_col=0)
    
#     gex.index = [x.replace('MT_', 'MT-').replace('pooled_', 'pooled-') for x in gex.index]
#     atac.index = [x.replace('MT_', 'MT-').replace('pooled_', 'pooled-') for x in atac.index]

#     gex['sample'] = [x.split('_')[-1] for x in gex.index]
#     atac['sample'] = [x.split('_')[-1] for x in atac.index]

#     gex = gex[gex['sample'] == s]
#     atac = atac[atac['sample'] == s]
    
#     print(gex.shape)
    
#     ### insanely ratchet lol
#     s = samples[i]
#     indir = f'{projdir}/demux/regular/var_consistency/gex/{s}/'
#     outdir = f'{projdir}/csv/var_consistency/gex/{s}'
    
#     steps = list(range(0, gex.shape[0], interval))
    
#     gex = gex.drop(columns=['sample'])
#     atac = atac.drop(columns=['sample'])
    
#     tmp_gex = gex.copy()
#     tmp_atac = atac.copy()
#     tmp_gex['inter'] = gex.eq(gex.iloc[:, 0], axis=0).all(1)
#     tmp_atac['inter'] = atac.eq(atac.iloc[:, 0], axis=0).all(1)
#     tmp_gex['majority'] = gex.mode(axis=1)[0]
#     tmp_atac['majority'] = atac.mode(axis=1)[0]

#     gex['majority'] = tmp_gex['majority']
#     atac['majority'] = tmp_atac['majority']

#     gex_inter = pd.DataFrame(tmp_gex[tmp_gex['inter']]['demuxlet'])
#     gex_inter.columns = ['inter']

#     atac_inter = pd.DataFrame(tmp_atac[tmp_atac['inter']]['demuxlet'])
#     atac_inter.columns = ['inter']

#     for cov in cov_thresh:
#         gex_con = read_varcon_steps(indir, steps, cov)
#         gex_con.index = [f'{x}_{s}' for x in gex_con.index]
#         gex_con.index = [x.replace('MT_', 'MT-').replace('pooled_', 'pooled-') for x in gex_con.index]
        
#         inter_con = gex_con[tmp_gex['inter']]
#         gex_inter_con = process_varcon(gex_inter, inter_con, donors, 'inter')
#         gex_inter_con.to_csv(f'{outdir}/inter_cov{cov}_varcon.csv', sep='\t', header=True, index=True)

#         for method in gex.columns:
# #             print(method)
#             consistency = process_varcon(gex, gex_con, donors, method)
#             consistency.to_csv(f'{outdir}/{method}_cov{cov}_varcon.csv', sep='\t', header=True, index=True)
#         inter_con = pd.DataFrame(gex_con[tmp_gex['inter']])
#         gex_inter_con = process_varcon(gex_inter, inter_con, donors, 'inter')    
#         gex_inter_con.to_csv(f'{outdir}/inter_cov{cov}_varcon.csv', sep='\t', header=True, index=True)
        
#     for cov in cov_thresh:
# #         print(method)
#         atac_con = read_varcon_steps(indir, steps, cov)
#         atac_con.index = [f'{x}_{s}' for x in atac_con.index]
#         atac_con.index = [x.replace('MT_', 'MT-').replace('pooled_', 'pooled-') for x in atac_con.index]
        
#         inter_con = atac_con[tmp_atac['inter']]
#         atac_inter_con = process_varcon(atac_inter, inter_con, donors, 'inter')
#         atac_inter_con.to_csv(f'{outdir}/inter_cov{cov}_varcon.csv', sep='\t', header=True, index=True)

#         for method in atac.columns:
#             consistency = process_varcon(atac, atac_con, donors, method)
#             consistency.to_csv(f'{outdir}/{method}_cov{cov}_varcon.csv', sep='\t', header=True, index=True)
#         inter_con = pd.DataFrame(atac_con[tmp_atac['inter']])
#         atac_inter_con = process_varcon(atac_inter, inter_con, donors, 'inter')
#         atac_inter_con.to_csv(f'{outdir}/inter_cov{cov}_varcon.csv', sep='\t', header=True, index=True)